In [105]:
# !pip install plotly==5.13.0
# !pip install dash
# !pip install wordcloud
# !pip install termcolor

In [106]:
# Data manipulation
import pandas as pd
import numpy as np

# Data visualization
import plotly
import plotly.express as px
import plotly.graph_objs as go #for interactive plots
from plotly.offline import init_notebook_mode, iplot #
import seaborn as sns

# World Cloud
from wordcloud import WordCloud, STOPWORDS

# Text color
from termcolor import colored

# Datetime
import datetime 

# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

## Loading the dataset

In [107]:
data = pd.read_csv("data/raw_data.csv", encoding='latin1')

## Looking at the dataset

In [108]:
print(data.shape)
data.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


**Observations:**

1. Our dataset has 541,909 rows and 8 columns.
2. Based on the source of the data, each of the columns in our dataset are described as below:

    `InvoiceNo`: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.  
    `StockCode`: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.  
    `Description`: Product (item) name. Nominal.  
    `Quantity`: The quantities of each product (item) per transaction. Numeric.  
    `InvoiceDate`: Invice Date and time. Numeric, the day and time when each transaction was generated.  
    `UnitPrice`: Unit price. Numeric, Product price per unit in sterling.  
    `CustomerID`: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.  
    `Country`: Country name. Nominal, the name of the country where each customer resides.    
    
    
3. There are multiple items linked to a single `InvoiceNo` i.e. items purchased by a customer in the same basket at a point of time. There are multiple Invoices linked to the same `CustomerID` i.e. different shopping instances by a customer during the period that the dataset covers.
4. We have columns `Quantity` and `Unit price` which will help us to understand the sales revenue.

In [109]:
print(colored("Information about the data", "red", attrs=['bold']))#using termcolor.colored module
data.info()

Information about the data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


**Observations:**

1. The dataset looks fairly clean. Only the `Description` and `CustomerID` columns have null values which isn't insubstantial. This isn't ideal. We will need to deal with this.
2. The `InvoiceDate`is an object data type. We will want to change this in our further analysis.
3. The `CustomerID` is a float data type which does't make much sense. So we will change this as well.

In [110]:
data.nunique() 

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

Unique values in each coloumn provide us a better understanding of our datatset. We see from the above that:

1. There are 4,372 unique Customers in our dataset.
2. There are 38 different countries in the dataset.

In [111]:
round(data["Country"].value_counts(normalize = True).sort_values(ascending = False),3)

United Kingdom          0.914
Germany                 0.018
France                  0.016
EIRE                    0.015
Spain                   0.005
Netherlands             0.004
Belgium                 0.004
Switzerland             0.004
Portugal                0.003
Australia               0.002
Norway                  0.002
Italy                   0.001
Channel Islands         0.001
Finland                 0.001
Cyprus                  0.001
Sweden                  0.001
Unspecified             0.001
Austria                 0.001
Denmark                 0.001
Japan                   0.001
Poland                  0.001
Israel                  0.001
USA                     0.001
Hong Kong               0.001
Singapore               0.000
Iceland                 0.000
Canada                  0.000
Greece                  0.000
Malta                   0.000
United Arab Emirates    0.000
European Community      0.000
RSA                     0.000
Lebanon                 0.000
Lithuania 

We see that the majority of our observations are linked to sales to customers in the UK with almost 92% of the entire dataset. I am interested in understanding the customers of the UK market so I will focus on that subset of the data in the remaining of the analysis.

In [112]:
data_uk = data[data["Country"] == "United Kingdom"].copy()

In [113]:
print(data_uk.shape)
print(data_uk["Country"].value_counts())

(495478, 8)
United Kingdom    495478
Name: Country, dtype: int64


Our dataset now has 495,478 observations relating to United Kingdom only.

## Cleaning the data

### 1.Duplicated observations

In [114]:
#Count duplicated observations across all columns
data_uk.duplicated().sum()

5178

In [115]:
#Drop observations which are duplicated across all columns
data_uk.drop_duplicates(inplace = True)

Our dataset now has 490,300 observations

In [116]:
data_uk.shape

(490300, 8)

### 2. Data types

In [117]:
data_uk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490300 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    490300 non-null  object 
 1   StockCode    490300 non-null  object 
 2   Description  488846 non-null  object 
 3   Quantity     490300 non-null  int64  
 4   InvoiceDate  490300 non-null  object 
 5   UnitPrice    490300 non-null  float64
 6   CustomerID   356728 non-null  float64
 7   Country      490300 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.7+ MB


We see that `InvoiceDate` and `CustomerID` are stored as string and numeric data types respectively. This isn't helpful for our analysis. So we will convert these.

In [118]:
#Converting InvoiceDate to datetime
data_uk["InvoiceDate"] = pd.to_datetime(data_uk.InvoiceDate)

#Converting CustomerID which are not missing to string
data_uk["CustomerID"] = data_uk["CustomerID"].astype(str).mask(data_uk["CustomerID"].isnull(), np.NaN)

In [119]:
data_uk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490300 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    490300 non-null  object        
 1   StockCode    490300 non-null  object        
 2   Description  488846 non-null  object        
 3   Quantity     490300 non-null  int64         
 4   InvoiceDate  490300 non-null  datetime64[ns]
 5   UnitPrice    490300 non-null  float64       
 6   CustomerID   356728 non-null  object        
 7   Country      490300 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.7+ MB


### 3.Missing values

In [120]:
null_percent = round((data_uk.isnull().sum() / data_uk.shape[0]) * 100,2)
print(colored("Percentage of missing Values", "red", attrs=['bold']))
null_percent.sort_values(ascending=False)
      

Percentage of missing Values


CustomerID     27.24
Description     0.30
InvoiceNo       0.00
StockCode       0.00
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
Country         0.00
dtype: float64

We see that more than a quarter of the `CustomerID` and 0.3% of `Description` are missing in our dataset. Let's delve into the rows with missing CustomerID first.

In [121]:
#Looking at the observations with null values for `CustomerID`
data_uk[data_uk['CustomerID'].isna()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


There doesn't seem to be anyything that stands out to be peculiar in the above except for index 622:
1. The null value in the `Description` column and
2. The 0 `UnitPrice` column

We will look if this is the case for other rows which have null values for `CustomerID`

In [122]:
df_null_customer_id = data_uk[data_uk['CustomerID'].isna()]

#Looking at rows which have null values in both the CustomerID and Description columns
null_df = df_null_customer_id[df_null_customer_id['Description'].isna()]
null_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,2011-12-07 18:26:00,0.0,NaN,United Kingdom
535326,581203,23406,NaN,15,2011-12-07 18:31:00,0.0,NaN,United Kingdom
535332,581209,21620,NaN,6,2011-12-07 18:35:00,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom


In [123]:
#Checking if any of the rows contain cancelled orders as given by a C in the InvoiceID
null_df[null_df["InvoiceNo"].str.contains(pat = 'C') == True]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [124]:
null_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,1454.0,-9.359697,243.238758,-3667.0,-24.0,-3.0,4.0,5568.0
UnitPrice,1454.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


So there are 1,454 observations in our dataset which has null values in both `Description` and `CustomerID` columns. Additionally, these observations also have a 0 unit price which seems odd. These observations look to be incorrect datapoints. I will drop these from our dataset.

In [125]:
#Dropping 1,454 observations
data_uk = data_uk.dropna(subset=["Description", "CustomerID"], how='all')

In [126]:
#Looking at percentage of null values across all columns in our dataset now
round((data_uk.isnull().sum() / data_uk.shape[0]) * 100,2).sort_values(ascending = False)

CustomerID     27.03
InvoiceNo       0.00
StockCode       0.00
Description     0.00
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
Country         0.00
dtype: float64

Our cleaning removed all the rows with missing values in the `Decription` column. But we still have significant missing values in `CustomerID` column. Let's take another look.

In [127]:
data_uk[data_uk['CustomerID'].isna()].describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,132118.0,2.036566,62.397120,-9600.00,1.00,1.00,3.00,4000.00
UnitPrice,132118.0,8.078911,152.644233,-11062.06,1.63,3.29,5.79,17836.46


Now we see that there are some negative values in both the `Quantity` and `UnitPrice` columns where `CustomerID` is missing. This looks odd as I would expect that both these columns should have positive values. Let's dig further into these observations.

In [128]:
#Subsetting the data to create a dataframe with null values for CustomerID
df_null_customer_id = data_uk[data_uk['CustomerID'].isna()]

df_null_customer_id[df_null_customer_id["Quantity"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
7313,537032,21275,?,-30,2010-12-03 16:50:00,0.00,NaN,United Kingdom
11502,C537251,22429,ENAMEL MEASURING JUG CREAM,-2,2010-12-06 10:45:00,4.25,NaN,United Kingdom
11503,C537251,22620,4 TRADITIONAL SPINNING TOPS,-8,2010-12-06 10:45:00,1.25,NaN,United Kingdom
11504,C537251,21890,S/6 WOODEN SKITTLES IN COTTON BAG,-2,2010-12-06 10:45:00,2.95,NaN,United Kingdom
11505,C537251,22564,ALPHABET STENCIL CRAFT,-5,2010-12-06 10:45:00,1.25,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,2011-12-07 18:36:00,0.00,NaN,United Kingdom
535335,581212,22578,lost,-1050,2011-12-07 18:38:00,0.00,NaN,United Kingdom
535336,581213,22576,check,-30,2011-12-07 18:38:00,0.00,NaN,United Kingdom
536908,581226,23090,missing,-338,2011-12-08 09:56:00,0.00,NaN,United Kingdom


We see that there are two types of `InvoiceNo` here. One which starts with the letter C and one which doesn't. The ones which start with a C are cancelled orders. These rows also differ from the others in terms on `Description`and `UnitPrice`. These are legitimate rows that will be helpful in future analysis. I will therefore subset my data into one where I exclude these rows.

In [129]:
#Dataframe of only non-cancelled orders

data_uk_non_cancelled = data_uk[data_uk["InvoiceNo"].str.contains(pat = 'C') == False]

#Dataframe of only cancelled orders

data_uk_cancelled = data_uk[data_uk["InvoiceNo"].str.contains(pat = 'C') == True]


In [130]:
#Looking at the dataset with non-cancelled orders only
data_uk_non_cancelled.describe()

,Quantity,UnitPrice
count,481025.000000,481025.000000
mean,9.406956,3.810877
std,166.723038,41.495292
min,-9600.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.100000
75%,10.000000,4.130000
max,80995.000000,13541.330000


In [131]:
#Looking at observations which have Quantity less than or equal to 0
data_uk_non_cancelled[data_uk_non_cancelled["Quantity"] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
7313,537032,21275,?,-30,2010-12-03 16:50:00,0.0,NaN,United Kingdom
13217,537425,84968F,check,-20,2010-12-06 15:35:00,0.0,NaN,United Kingdom
13218,537426,84968E,check,-35,2010-12-06 15:36:00,0.0,NaN,United Kingdom
13264,537432,35833G,damages,-43,2010-12-06 16:10:00,0.0,NaN,United Kingdom
21338,538072,22423,faulty,-13,2010-12-09 14:10:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,2011-12-07 18:36:00,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,2011-12-07 18:38:00,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,2011-12-07 18:38:00,0.0,NaN,United Kingdom
536908,581226,23090,missing,-338,2011-12-08 09:56:00,0.0,NaN,United Kingdom


In [132]:
data_uk_non_cancelled[data_uk_non_cancelled["Quantity"] <= 0].describe()

,Quantity,UnitPrice
count,474.000000,474.0
mean,-339.242616,0.0
std,920.343402,0.0
min,-9600.000000,0.0
25%,-256.000000,0.0
50%,-72.000000,0.0
75%,-28.000000,0.0
max,-1.000000,0.0


In [133]:
pd.options.display.max_rows = 400 #increases the viewing pane of my dataset

data_uk_non_cancelled[data_uk_non_cancelled["Quantity"] <= 0].Description.value_counts()

check                                  120
damages                                 45
damaged                                 42
?                                       41
sold as set on dotcom                   20
Damaged                                 14
thrown away                              9
Unsaleable, destroyed.                   9
??                                       7
wet damaged                              5
damages?                                 5
ebay                                     5
smashed                                  4
missing                                  3
wet pallet                               3
CHECK                                    3
sold as 1                                2
incorrect stock entry.                   2
crushed                                  2
adjustment                               2
wet/rusty                                2
reverse 21/5/10 adjustment               2
?missing                                 2
damages wax

**Observations:**

1. From the non-cancelled observations, we saw that there were rows with negative or zero values for `Quantity` and `UnitPrice`. These seem odd.
2. Looking deeper into these odd quantities within this dataset, we see that all the `CustomersID` are missing and `UnitPrice` equals 0.
3. Looking deeper into these rows, they seem to be of a particular type given what is recorded in the `Description`. The main insight coming from these are that these seem to be some form inventory related data that doesn't involve sales. Thus it makes sense to not have an `CustomerID` associated with these rows. We will drop these 474 rows. 

In [134]:
#Dropping 474 observations where Quantity value is negative and UnitPrice is 0.

data_uk_non_cancelled = data_uk_non_cancelled.drop(data_uk_non_cancelled[data_uk_non_cancelled["Quantity"] <= 0].index)

In [135]:
#Looking at rows with UnitPrice equals 0
data_uk_non_cancelled[data_uk_non_cancelled["UnitPrice"] == 0]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
6391,536941,22734,amazon,20,2010-12-03 12:08:00,0.0,NaN,United Kingdom
6392,536942,22139,amazon,15,2010-12-03 12:08:00,0.0,NaN,United Kingdom
14335,537534,85064,CREAM SWEETHEART LETTER RACK,1,2010-12-07 11:48:00,0.0,NaN,United Kingdom
14336,537534,84832,ZINC WILLIE WINKIE CANDLE STICK,1,2010-12-07 11:48:00,0.0,NaN,United Kingdom
14337,537534,84692,BOX OF 24 COCKTAIL PARASOLS,2,2010-12-07 11:48:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
524622,580609,22927,Amazon,1,2011-12-05 11:41:00,0.0,NaN,United Kingdom
535325,581202,23404,check,41,2011-12-07 18:30:00,0.0,NaN,United Kingdom
535334,581211,22142,check,14,2011-12-07 18:36:00,0.0,NaN,United Kingdom
538504,581406,46000M,POLYESTER FILLER PAD 45x45cm,240,2011-12-08 13:58:00,0.0,NaN,United Kingdom


In [136]:
data_uk_non_cancelled[data_uk_non_cancelled["UnitPrice"] == 0].Description.value_counts()

check                                  39
found                                  25
adjustment                             14
FRENCH BLUE METAL DOOR SIGN 1           9
FRENCH BLUE METAL DOOR SIGN 8           8
Found                                   8
amazon                                  8
RECIPE BOX PANTRY YELLOW DESIGN         7
OWL DOORSTOP                            7
FRENCH BLUE METAL DOOR SIGN 4           7
FRENCH BLUE METAL DOOR SIGN No          7
FRENCH BLUE METAL DOOR SIGN 3           7
Amazon                                  7
FRENCH BLUE METAL DOOR SIGN 7           6
FRENCH BLUE METAL DOOR SIGN 5           6
?                                       6
RED KITCHEN SCALES                      6
FRENCH BLUE METAL DOOR SIGN 6           6
RED RETROSPOT CHARLOTTE BAG             5
DOORMAT WELCOME TO OUR HOME             5
FRENCH BLUE METAL DOOR SIGN 2           5
RECIPE BOX BLUE SKETCHBOOK DESIGN       5
had been put aside                      5
MINT KITCHEN SCALES               

In [137]:
##Looking at rows with UnitPrice less than 0
data_uk_non_cancelled[data_uk_non_cancelled["UnitPrice"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


**Observations:**
1. All the observations with either 0 or negative values for `UnitPrice` have missing values for `CustomerID`
2. The rows with negative `UnitPrice` appear to be some adjustments rather than sales data. So we will drop these two observations.
3. The rows with 0 `UnitPrice` is a mixture of inventory adjustments and products with valid `Description`. However it doesn't make sense why the `UnitPrice` of a product will be 0 . Without further insights it wouldn't behelpful to include these in further analysis. So we will drop these 564 rows.

In [138]:
#Dropping 566 observations
data_uk_non_cancelled = data_uk_non_cancelled.drop(data_uk_non_cancelled[data_uk_non_cancelled["UnitPrice"] <= 0].index)

In [139]:
#Looking at the dataset with cancelled orders only
data_uk_cancelled.describe()

,Quantity,UnitPrice
count,7821.000000,7821.000000
mean,-33.377317,51.159761
std,1248.444349,718.905804
min,-80995.000000,0.010000
25%,-6.000000,1.450000
50%,-2.000000,2.950000
75%,-1.000000,5.950000
max,-1.000000,38970.000000


In [140]:
data_uk_cancelled[data_uk_cancelled["Quantity"] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


The dataset with cancelled orders has negative values for `Quantity` but valid values for `CustomerID`. These seem to be basically quantities which have been returned by the customers. So we will keep these observations.

In [141]:
#Looking at percentage of null values across all columns in our dataset of non-cancelled orders 
round((data_uk_non_cancelled .isnull().sum() / data_uk_non_cancelled .shape[0]) * 100,2).sort_values(ascending = False)

CustomerID     27.25
InvoiceNo       0.00
StockCode       0.00
Description     0.00
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
Country         0.00
dtype: float64

In [142]:
data_uk_non_cancelled [data_uk_non_cancelled ['CustomerID'].isna()].describe()

,Quantity,UnitPrice
count,130782.000000,130782.000000
mean,3.101352,6.249372
std,11.168277,59.902530
min,1.000000,0.060000
25%,1.000000,1.630000
50%,1.000000,3.290000
75%,3.000000,5.790000
max,1820.000000,13541.330000


In [143]:
#Looking at percentage of null values across all columns in our dataset of cancelled orders 
round((data_uk_cancelled .isnull().sum() / data_uk_cancelled .shape[0]) * 100,2).sort_values(ascending = False)

CustomerID     4.09
InvoiceNo      0.00
StockCode      0.00
Description    0.00
Quantity       0.00
InvoiceDate    0.00
UnitPrice      0.00
Country        0.00
dtype: float64

In [144]:
data_uk_cancelled [data_uk_cancelled ['CustomerID'].isna()].describe()

,Quantity,UnitPrice
count,320.000000,320.000000
mean,-5.865625,850.588500
std,15.684709,2587.851096
min,-144.000000,0.420000
25%,-1.000000,2.950000
50%,-1.000000,20.510000
75%,-1.000000,238.852500
max,-1.000000,17836.460000


In [145]:
print(data_uk_non_cancelled.shape)
data_uk_non_cancelled[data_uk_non_cancelled["CustomerID"].isnull()]

(479985, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


In [146]:
print(data_uk_cancelled.shape)
data_uk_cancelled[data_uk_cancelled["CustomerID"].isnull()]

(7821, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
11502,C537251,22429,ENAMEL MEASURING JUG CREAM,-2,2010-12-06 10:45:00,4.25,NaN,United Kingdom
11503,C537251,22620,4 TRADITIONAL SPINNING TOPS,-8,2010-12-06 10:45:00,1.25,NaN,United Kingdom
11504,C537251,21890,S/6 WOODEN SKITTLES IN COTTON BAG,-2,2010-12-06 10:45:00,2.95,NaN,United Kingdom
11505,C537251,22564,ALPHABET STENCIL CRAFT,-5,2010-12-06 10:45:00,1.25,NaN,United Kingdom
11506,C537251,21891,TRADITIONAL WOODEN SKIPPING ROPE,-3,2010-12-06 10:45:00,1.25,NaN,United Kingdom
11507,C537251,22747,POPPY'S PLAYHOUSE BATHROOM,-6,2010-12-06 10:45:00,2.10,NaN,United Kingdom
11508,C537251,22454,MEASURING TAPE BABUSHKA RED,-8,2010-12-06 10:45:00,2.95,NaN,United Kingdom
11509,C537251,22327,ROUND SNACK BOXES SET OF 4 SKULLS,-4,2010-12-06 10:45:00,2.95,NaN,United Kingdom
11510,C537251,21915,RED HARMONICA IN BOX,-4,2010-12-06 10:45:00,1.25,NaN,United Kingdom
11511,C537251,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9,2010-12-06 10:45:00,2.55,NaN,United Kingdom


We see from the above that both our cancelled and non-cancelled datasets have values in the `StockCode` column which don't represent any inventory per se. This can be expected irrespective of the `CustomerID` having a missing value or not. As these observations don't help with our analysis, we will drop these from both datasets.

In [147]:
#Dropping irrelevant stock codes from the non-cancelled orders dataset.
df1 = data_uk_non_cancelled[~data_uk_non_cancelled['StockCode'].str.contains('[^A-Za-z\s]')]
data_uk_non_cancelled = data_uk_non_cancelled.drop(df1.index)

#Dropping irrelevant stock codes from the cancelled orders dataset.
df2 = data_uk_cancelled[~data_uk_cancelled['StockCode'].str.contains('[^A-Za-z\s]')]
data_uk_cancelled = data_uk_cancelled.drop(df2.index)


At this point, we have cleaned our datasets to avoid any common and apparent anomalies in the data. Although we still have null values in the `CustomerID` column, this looks like more of a data entry issue rather than data anomaly. So we will keep these observations for further analysis.

In [148]:
print("Number of observations in dataset of non-cancelled orders :",data_uk_non_cancelled.shape[0])
print("Number of observations in dataset of cancelled orders :",data_uk_cancelled.shape[0])

Number of observations in dataset of non-cancelled orders : 478908
Number of observations in dataset of cancelled orders : 7329


## Exploring the dataset of non-cancelled orders

In [149]:
#What are the total number of unique customers who purchased on our website in the period covered?
print(colored("Total number of customers", 
              "red", attrs=['bold']) 
                + "\n" 
                + str(data_uk_non_cancelled["CustomerID"].nunique()))

Total number of customers
3916


In [150]:
#What is the total number of products sold?
print(colored("Total number of Products", 
              "red", attrs=['bold']) 
              + "\n" 
              + str(data_uk_non_cancelled["StockCode"].nunique()))

Total number of Products
3905


In [168]:
#Which products are most popular?
most_sold = data_uk_non_cancelled.groupby(['StockCode'])['Quantity'].sum().reset_index()
most_sold = most_sold.sort_values(by=["Quantity"], ascending=False)

# Plotly bar plot
fig = px.bar(most_sold.head(10), x='StockCode', y='Quantity', 
              text_auto='.2s',
             title = "Best Selling Item Stock Code ")
fig.update_yaxes(showgrid=False)
fig.update_xaxes(showgrid=False)
fig.update_layout(plot_bgcolor= 'white')
fig.update_traces(marker_color='darkred')

fig.show()

In [152]:
display(data_uk_non_cancelled[data_uk_non_cancelled["StockCode"] == "23843"].head(1))
display(data_uk_non_cancelled[data_uk_non_cancelled["StockCode"] == "23166"].head(1))
display(data_uk_non_cancelled[data_uk_non_cancelled["StockCode"] == "22197"].head(1))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
221,536390,22197,SMALL POPCORN HOLDER,100,2010-12-01 10:19:00,0.72,17511.0,United Kingdom


Basket size refers to the total number of products a customer purchases in one transaction. It is a measurement that describes how many items a customer purchases in a single transaction.

In [153]:
print(colored("Total number of transactions", "red", attrs=['bold']) + "\n" + str(data_uk_non_cancelled["InvoiceNo"].nunique()))

Total number of transactions
17901


In [154]:
#What is the basket size of the transactions?
basket_size = data_uk_non_cancelled.groupby(['InvoiceNo'])['Quantity'].sum().reset_index()
basket_size = basket_size.sort_values(by=["Quantity"], ascending=False)

basket_size 

,InvoiceNo,Quantity
17872,581483,80995
1942,541431,74215
15176,574941,14149
15766,576365,13956
12191,567423,12572
...,...,...
3693,546002,1
10339,562588,1
3660,545900,1
13714,571256,1


In [155]:
display(data_uk_non_cancelled[data_uk_non_cancelled["InvoiceNo"]== "581483"].head(3))
display(data_uk_non_cancelled[data_uk_non_cancelled["InvoiceNo"]== "541431"].head(3))
display(data_uk_non_cancelled[data_uk_non_cancelled["InvoiceNo"]== "574941"].head(3))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
446951,574941,23582,VINTAGE DOILY JUMBO BAG RED,147,2011-11-07 17:42:00,4.95,NaN,United Kingdom
446952,574941,23581,JUMBO BAG PAISLEY PARK,314,2011-11-07 17:42:00,4.95,NaN,United Kingdom
446953,574941,23570,TRADITIONAL PICK UP STICKS GAME,384,2011-11-07 17:42:00,2.95,NaN,United Kingdom


In [156]:
#Segmenting the transactions based on size of the basket
basket_size_outlier = basket_size.loc[basket_size["Quantity"] > 2000]
basket_size_bulk = basket_size.loc[(basket_size["Quantity"] > 100) & (basket_size["Quantity"] <= 2000)]
basket_size_normal = basket_size.loc[basket_size["Quantity"] <=100]

print("Number of transactions with more than 2000 items:", len(basket_size_outlier))
print("Number of transactions with more than 100 items but less than or wqual to 2000 items:", len(basket_size_bulk))
print("Number of transactions with less than or equal to 100 items:",len(basket_size_normal))

Number of transactions with more than 2000 items: 154
Number of transactions with more than 100 items but less than or wqual to 2000 items: 11215
Number of transactions with less than or equal to 100 items: 6532


In [157]:
#Average basket size

avg_basket_size_normal = round(basket_size_normal["Quantity"].sum()/basket_size_normal["InvoiceNo"].count())
avg_basket_size_bulk = round(basket_size_bulk["Quantity"].sum()/basket_size_bulk["InvoiceNo"].count())

print("Average basket size of normal transactions:", avg_basket_size_normal)
print("Average basket size of bulk transactions:", avg_basket_size_bulk)

Average basket size of normal transactions: 48
Average basket size of bulk transactions: 322


In [158]:
fig = px.histogram(basket_size_bulk, x="Quantity",nbins =20,color_discrete_sequence = ["darkred"])
fig.update_layout(plot_bgcolor= 'white')
fig.show()

In [159]:
fig = px.histogram(basket_size_normal, x="Quantity", color_discrete_sequence =["darkcyan"],nbins =20)
fig.update_layout(plot_bgcolor= 'white')
fig.show()

**Observations:**

1. It looks like among baskets which don't involve bulk transactions, the most common basket size is 1-4 items. When looking at bulky orders, the most common basket size is 101-149 items. 
2. The average basket size of a normal transaction is 48 items whereas the avearge basket size of a bulk transaction is 322 items

In [161]:
#What do the number of transactions look like per month?

#Taking the month out of InvoiceDate
data_uk_non_cancelled["Month"] = data_uk_non_cancelled['InvoiceDate'].dt.strftime('%B')


invoice_basket_size = data_uk_non_cancelled.groupby(['Month', 'InvoiceNo']).size().reset_index(name = "BasketSize")
monthly_transactions = invoice_basket_size.groupby(['Month'])['InvoiceNo'].count().reset_index(name = "#Transactions")

In [162]:
invoice_basket_size

,Month,InvoiceNo,BasketSize
0,April,548550,12
1,April,548551,18
2,April,548552,10
3,April,548554,19
4,April,548555,15
...,...,...,...
17896,September,569145,2
17897,September,569147,20
17898,September,569148,30
17899,September,569149,2


In [163]:
monthly_transactions

,Month,#Transactions
0,April,1148
1,August,1195
2,December,2184
3,February,990
4,January,967
5,July,1317
6,June,1379
7,March,1302
8,May,1518
9,November,2481


In [164]:
order_months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
monthly_transactions = monthly_transactions.sort_values('Month', key = lambda x : pd.Categorical(x, categories=order_months, ordered=True))
plot_title = "Monthly Total Transactions"

In [165]:
# Plotly line chart 
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_transactions["Month"], y=monthly_transactions["#Transactions"], name='#Transactions',
                         line=dict(color="purple", width=3)))

fig.update_layout(plot_bgcolor= 'white')

fig.update_layout(title_text=plot_title, 
                  title_font=dict(size=20, color="purple", family="Arial"))

fig.show()

In [166]:
#What is the average basket size per month?

monthly_basket_size = invoice_basket_size.groupby(['Month'])['BasketSize'].mean().reset_index(name = "Average_BasketSize")
monthly_basket_size["monthly_basket_size"]= round(monthly_basket_size["Average_BasketSize"])
monthly_basket_size = monthly_basket_size.sort_values('Month', key = lambda x : pd.Categorical(x, categories=order_months, ordered=True))
plot_title = "Monthly Average Basket size"

In [167]:
# Plotly line chart 
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_basket_size["Month"], y=monthly_basket_size["Average_BasketSize"], name='Average Basket Size',
                         line=dict(color="forestgreen", width=3)))

fig.update_layout(plot_bgcolor= 'white')

fig.update_layout(title_text=plot_title, 
                  title_font=dict(size=20, color="forestgreen", family="Arial"))

fig.show()

In [58]:
#What does the monthly sales revenue overall look like?

#Calculating revenue column
data_uk_non_cancelled["Revenue"] = data_uk_non_cancelled["UnitPrice"] * data_uk_non_cancelled["Quantity"]



In [59]:
month_sales = data_uk_non_cancelled.groupby('Month')['Revenue'].sum().reset_index()
month_sales = month_sales.sort_values('Month', key = lambda x : pd.Categorical(x, categories=order_months, ordered=True))
plot_title = "Monthly Total Revenue"

In [60]:
# Plotly line chart 
fig = go.Figure()

fig.add_trace(go.Scatter(x=month_sales["Month"], y=month_sales["Revenue"], name='Revenue',
                         line=dict(color="darkred", width=3)))

fig.update_layout(plot_bgcolor= 'white')

fig.update_layout(title_text=plot_title, 
                  title_font=dict(size=20, color="darkred", family="Arial"))

fig.show()

In [61]:
#What does the monthly sale of a particular product look like?

stock_code = input("Enter Stock Code: ")

invoice_product = data_uk_non_cancelled[data_uk_non_cancelled["StockCode"] == stock_code]
month_sales = invoice_product.groupby('Month')['Revenue'].sum().reset_index()
month_sales = month_sales.sort_values('Month', key = lambda x : pd.Categorical(x, categories=order_months, ordered=True))
plot_title = "Monthly Revenue of product: {} ".format(stock_code)


Enter Stock Code: 22197


In [62]:
# Plotly line chart 
fig = go.Figure()

fig.add_trace(go.Scatter(x=month_sales["Month"], y=month_sales["Revenue"], name='Revenue',
                         line=dict(color="darkcyan", width=3)))
fig.update_layout(plot_bgcolor= 'white')

fig.update_layout(title_text=plot_title, 
                  title_font=dict(size=20, color="darkcyan", family="Arial"))

fig.show()

In [90]:
#Who are our most loyal customers?

top_customer_all = data_uk_non_cancelled.groupby(['CustomerID'])['Quantity', 'Revenue'].sum().reset_index()
top_customer_sorted_q = top_customer_all.sort_values(by=["Quantity"], ascending=False)
top_customer_sorted_r = top_customer_all.sort_values(by=["Revenue"], ascending=False)

top_customer_quantity = top_customer_sorted_q.head(10)
top_customer_revenue = top_customer_sorted_r.head(10)


In [97]:
fig = px.bar(top_customer_quantity , x="Quantity", y="CustomerID", 
             orientation='h', 
             title='Top customers by Quantity purchased')
fig.update_yaxes(autorange="reversed", showgrid=False)
fig.update_xaxes(showgrid=False)
fig.update_layout(plot_bgcolor= 'white')
fig.update_traces(marker_color='darkcyan')

fig.show()

In [98]:
fig = px.bar(top_customer_revenue , x="Revenue", y="CustomerID", 
             orientation='h', 
             title='Top customers by Revenue generated')
fig.update_yaxes(autorange="reversed", showgrid=False)
fig.update_xaxes(showgrid=False)
fig.update_layout(plot_bgcolor= 'white')
fig.update_traces(marker_color='darkred')

fig.show()

In [174]:
#Which products are most cancelled?

data_uk_cancelled['Quantity'] = data_uk_cancelled['Quantity'] * (-1)

#Which products are most popular?
most_cancelled= data_uk_cancelled.groupby(['StockCode'])['Quantity'].sum().reset_index()
most_cancelled = most_cancelled.sort_values(by=["Quantity"], ascending=False)

# Plotly bar plot
fig = px.bar(most_cancelled.head(10), x='StockCode', y='Quantity', 
              text_auto='.2s',
             title = "Most cancelled Item Stock Code ")
fig.update_yaxes(showgrid=False)
fig.update_xaxes(showgrid=False)
fig.update_layout(plot_bgcolor= 'white')
fig.update_traces(marker_color='darkcyan')

fig.show()

In [178]:
display(data_uk_cancelled[data_uk_cancelled["StockCode"]== "84347"].head(1))
display(data_uk_cancelled[data_uk_cancelled["StockCode"]== "21108"].head(1))
display(data_uk_cancelled[data_uk_cancelled["StockCode"]== "85123A"].head(1))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
4287,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
160145,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-04-18 13:08:00,2.1,15749.0,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
14529,C537602,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1,2010-12-07 12:45:00,2.55,17511.0,United Kingdom
